In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
#from zipfile import ZipFile
#train_name = '/content/drive/MyDrive/DATA_2040/Molecular Exploration/Data/SMILES_train_imgs.zip'
#test_name = '/content/drive/MyDrive/DATA_2040/Molecular Exploration/Data/SMILES_test_imgs.zip'

# with ZipFile(train_name, 'r') as zip:

#   print("Extracting new train files")
#   zip.extractall()
#   print("done!")

# with ZipFile(test_name, 'r') as zip:

#   print("Extracting new test files")
#   zip.extractall()
#   print("done!")

#!unzip -uq "/content/drive/MyDrive/DATA_2040/Molecular Exploration/Data/SMILES_train_imgs.zip" -d "/content/drive/MyDrive/DATA_2040/Molecular Exploration/Data/SMILES_train_imgs"

In [4]:
!pip install -q wandb
import wandb

wandb.init(project="SMILES_CNN_v2")

wandb: Currently logged in as: jgreen (use `wandb login --relogin` to force relogin)


In [5]:
import pandas as pd

train_labels = pd.read_csv('/content/drive/MyDrive/Molecular Exploration/Data/label_mapping_train.csv', index_col='Unnamed: 0')
test_labels = pd.read_csv('/content/drive/MyDrive/Molecular Exploration/Data/label_mapping_test.csv', index_col = 'Unnamed: 0')

In [6]:
train_labels.head()
test_labels.head()

,file,target,smiles
0,0.png,0,CNc1ncnc2c1ncn2[C@@H]1O[C@H](CO)C(O)[C@H]1O
1,1.png,1,Oc1cc(O)cc(/C=C/c2ccc(O)c(O)c2)c1
2,2.png,0,COc1ccc2c(c[n+](C)c3c4cc5c(cc4ccc23)OCO5)c1OC....
3,3.png,0,Br.Cc1onc(O)c1CC(N)C(=O)O
4,4.png,0,Nc1nc(N)c(N=O)c(OCC2CCCCC2)n1


In [7]:
from PIL import Image
# Open the image form working directory
image = Image.open('/content/drive/MyDrive/Molecular Exploration/Data/SMILES_train_imgs/0.png')
# summarize some details about the image

In [8]:
image.size

(300, 300)

In [9]:
IMG_SIZE = image.size
BATCH_SIZE = 64

In [10]:
import os
import tensorflow as tf

train_imgs = "/content/drive/MyDrive/Molecular Exploration/Data/SMILES_train_imgs/"
test_imgs = "/content/drive/MyDrive/Molecular Exploration/Data/SMILES_test_imgs/"

# for filename in os.listdir(train_imgs):
#     if filename.endswith(".jpg") or filename.endswith(".png"):
#         print(os.path.join(directory, filename))
#     else:
#         continue

data_gen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)

labels = train_labels
labels['target']=labels['target'].astype(str)

train_generator = data_gen.flow_from_dataframe(dataframe=labels, 
                                               directory=train_imgs,
                                               subset='training',
                                               x_col='file',
                                               y_col='target',
                                               target_size=IMG_SIZE,
                                               batch_size=BATCH_SIZE,
                                               class_mode='categorical')

valid_generator = data_gen.flow_from_dataframe(dataframe=labels, 
                                               directory=train_imgs,
                                               subset='validation',
                                               x_col='file',
                                               y_col='target',
                                               target_size=IMG_SIZE,
                                               batch_size=BATCH_SIZE,
                                               class_mode='categorical')

labels = test_labels
labels['target']=labels['target'].astype(str)

test_generator = data_gen.flow_from_dataframe(dataframe=labels, 
                                               directory=test_imgs,
                                               subset=None,
                                               x_col='file',
                                               y_col='target',
                                               target_size=IMG_SIZE,
                                               batch_size=BATCH_SIZE,
                                               class_mode='binary')

Found 4634 validated image filenames belonging to 2 classes.
Found 1158 validated image filenames belonging to 2 classes.
Found 268 validated image filenames belonging to 2 classes.


In [13]:
from tensorflow.keras.layers import Input
from tensorflow.keras.applications import *

base = InceptionResNetV2(
    weights='imagenet',
    include_top = False,
    input_tensor = None,
    input_shape = (IMG_SIZE[0],IMG_SIZE[1],3)
)

In [14]:
def build_model():
    inputs = base.output
    model = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    model = tf.keras.layers.Dense(2, activation = 'softmax')(model)
    return model

callbacks = [wandb.keras.WandbCallback()]
 
model = tf.keras.Model(inputs=base.input, outputs=build_model())

In [15]:
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['AUC', 'accuracy'])

history = model.fit(train_generator,
                    validation_data=valid_generator,
                    epochs=30,
                    verbose=1,
                    callbacks=callbacks)


Epoch 1/30
73/73 [==============================] - 105s 1s/step - loss: 0.1783 - auc: 0.9785 - accuracy: 0.9644 - val_loss: 141.2323 - val_auc: 0.2433 - val_accuracy: 0.2427
Epoch 2/30
73/73 [==============================] - 81s 1s/step - loss: 0.0984 - auc: 0.9848 - accuracy: 0.9799 - val_loss: 3.1246 - val_auc: 0.7647 - val_accuracy: 0.7573
Epoch 3/30
73/73 [==============================] - 81s 1s/step - loss: 0.0916 - auc: 0.9888 - accuracy: 0.9800 - val_loss: 2.1813 - val_auc: 0.5266 - val_accuracy: 0.5553
Epoch 4/30
73/73 [==============================] - 81s 1s/step - loss: 0.0964 - auc: 0.9866 - accuracy: 0.9797 - val_loss: 0.8909 - val_auc: 0.7632 - val_accuracy: 0.7573
Epoch 5/30
73/73 [==============================] - 81s 1s/step - loss: 0.0858 - auc: 0.9923 - accuracy: 0.9791 - val_loss: 1.8484 - val_auc: 0.7634 - val_accuracy: 0.7573
Epoch 6/30
73/73 [==============================] - 81s 1s/step - loss: 0.0938 - auc: 0.9879 - accuracy: 0.9796 - val_loss: 0.6728 - val_

In [16]:
wandb.finish()

#model.save("/content/drive/MyDrive/DATA_2040/Molecular Exploration/model_checkpoints/SMILES_CNN.hdf5")

epoch,29
loss,4e-05
auc,1.0
accuracy,1.0
val_loss,3.89688
val_auc,0.77138
val_accuracy,0.76252
_runtime,2480
_timestamp,1618873434
_step,29
best_val_loss,0.6478


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▆▆▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁
auc,▁▂▃▃▄▄▅▅▆▆▇▇▇▇████████████████
accuracy,▁▂▂▂▂▂▂▂▂▃▃▃▄▄▄▆▆▆▇▄▆▇████████
val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_auc,▁█▅███████████████████████████
val_accuracy,▁█▅████████▇████▇█████████████
_runtime,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
